In [19]:
import nltk
from nltk.corpus import wordnet as wn
import json
import polars as pl

# Download WordNet data
nltk.download('wordnet')

import sys
sys.path.append('..')
from utils import utils as u

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Volpe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
def get_genus(sentence):
    genus = []

    comma_split = sentence.split(',')

    splitted_sent = []

    for cs in comma_split:
        if cs.split('.') != '':
            splitted_sent += cs.split('.')

    if '' in splitted_sent:
        splitted_sent.remove('')

    # Pattern di identificazione del genus
    targets = {
        1: [["it"], ["is", "'s"], ["a", "an", "the", "for"]],
        2: [["that", "of", "to"]],
        3: [["a", "an", "for", "to"]]
    }

    for sentence in splitted_sent:
        tokenized_def = u.get_lemmatized_tokens_list_pos(u.tokenizer(sentence))

        for i in range(0, len(tokenized_def)):
            if i > 0 and tokenized_def[i][0].lower() in targets[2][0]:
                if tokenized_def[i-1][0] not in genus and tokenized_def[i-1][1] == 'n':
                    genus.append(tokenized_def[i-1][0])

            if (i < len(tokenized_def)-1):
                if tokenized_def[i][1] == 'a' and tokenized_def[i+1][1] == 'n' and tokenized_def[i+1][0] not in genus:
                    genus.append(tokenized_def[i+1][0])

            if i < len(tokenized_def)-2 and tokenized_def[i][0].lower() in targets[3][0]:
                if tokenized_def[i+1][0] not in genus and tokenized_def[i+1][1] == 'n':
                    genus.append(tokenized_def[i+1][0])

                elif tokenized_def[i+1][1] == 'a' and tokenized_def[i+2][1] == 'n':
                    genus.append(tokenized_def[i+2][0])

            if (i < len(tokenized_def)-3):
                if tokenized_def[i][0].lower() in targets[1][0] and (tokenized_def[i+1][0] in targets[1][1]) and (tokenized_def[i+2][0] in targets[1][2]):
                    if tokenized_def[i+3][0] not in genus:
                        genus.append(tokenized_def[i+3][0])
    return genus


In [25]:
get_genus("A construction used to divide two rooms, temporarily closing the passage between them")

['construction']

In [21]:
def get_target_term(sentence):
    genus = get_genus(sentence)

    #print(genus)

    # prendo gli iponimi dei sensi del genus
    hy = []
    for g in genus:
        genus_synset = wn.synsets(u.lemmatize_word(g, 'n'))
        # print(genus_synset)
        for syns in genus_synset:
            for h in syns.hyponyms():
                hy.append(h)

    #print(hy)

    # vedo tra gli esempi quali sostantivi sono ricorrenti, e ne ricavo il synset
    results = {}
    for h in hy:
        # prendo gli esempi
        h_examples = h.examples() + [h.definition()]
        # print(h_examples)

        if len(h_examples) > 0:
            for ex in h_examples:
                if len(ex) > 1:
                    for token in u.noise_reduction_en_pos(ex):

                        if token[1] == 'n':
                            if token[0] not in results:
                                results[token[0]] = 1
                            else:
                                results[token[0]] += 1
    if len(results) == 0:
        return None

    
    
    #print(average_value)

    results = sorted(results.items(), key=lambda item: item[1], reverse=True)
    risultato = {chiave: valore for chiave,
                 valore in results if valore > 1}
    total_values = sum(risultato.values())
    if len(risultato) == 0:
        return None
    average_value = total_values/len(risultato)
    risultato = {chiave: valore for chiave,
                 valore in results if valore > average_value}

    return risultato


In [22]:
get_target_term("A construction used to divide two rooms, temporarily closing the passage between them")

{'structure': 58,
 'building': 11,
 'construction': 11,
 'wall': 5,
 'ship': 5,
 'room': 5,
 'part': 5,
 'people': 5}

In [23]:
df = json.loads(pl.read_csv(
    "../datasets/TLN-definitions-23.tsv", separator='\t').write_json())

data = {}

for col in df['columns']:
    if col['name'] != '1':
        data[col['name']] = []
        for v in col['values']:
            data[col['name']].append(v)
            

In [24]:
final_results = {}

for word in data:
    i = 0
    for sentence in data[word]:
        target_terms = get_target_term(sentence)
        if target_terms is not None:
            if word in target_terms:
                i += 1
    final_results[word] = i/len(data[word])

final_results


{'door': 0.23333333333333334,
 'ladybug': 0.0,
 'pain': 0.06666666666666667,
 'blurriness': 0.0}